TEXT-1

In [2]:
# Simple Login Program

# Predefined username and password
username = "admin"
password = "12345"

# Get user input
user_input = input("Enter username: ")
pass_input = input("Enter password: ")

# Check login credentials
if user_input == username and pass_input == password:
    print("Login successful!")
else:
    print("Invalid username or password.")


Enter username: rakshith
Enter password: 56789
Invalid username or password.


TEXT-2

In [8]:
# fetch_user_with_fallback.py
import re
import mysql.connector
from mysql.connector import Error as MySQLError
import sqlite3
import sys

DB_CONFIG = {
    "host": "localhost",
    "user": "dbuser",       # change or use root for quick docker test
    "password": "dbpass",   # change
    "database": "mydatabase",
    "port": 3306,
    "connection_timeout": 5,
}

USERNAME_RE = re.compile(r'^[A-Za-z0-9_]{3,30}$')

def validate_username(username: str) -> bool:
    return bool(USERNAME_RE.match(username))

def fetch_user_mysql(username: str):
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        if not conn.is_connected():
            print("MySQL: connected flag is False after connect()")
            return None

        cur = conn.cursor(dictionary=True)
        query = "SELECT id, username, email FROM users WHERE username = %s"
        cur.execute(query, (username,))
        user = cur.fetchone()
        cur.close()
        conn.close()
        return user

    except MySQLError as e:
        # MySQL connection/query failed — surface a helpful error
        print(f"Error connecting to MySQL: {e}")
        print("Please ensure a MySQL server is running and accessible with the provided credentials.")
        return None

def fetch_user_sqlite_demo(username: str):
    # In-memory SQLite demo (no external dependencies)
    conn = sqlite3.connect(":memory:")
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE,
            email TEXT
        )
    """)
    # Insert demo data
    cur.executemany("INSERT INTO users (username, email) VALUES (?, ?)",
                    [("alice", "alice@example.com"), ("bob", "bob@example.com")])
    conn.commit()

    cur.execute("SELECT id, username, email FROM users WHERE username = ?", (username,))
    row = cur.fetchone()
    if row:
        return dict(row)
    return None

def fetch_user(username: str):
    if not validate_username(username):
        raise ValueError("Invalid username format. Use 3-30 letters, numbers, or underscores.")

    # Try MySQL first
    user = fetch_user_mysql(username)
    if user is not None:
        return user

    # If MySQL failed or returned nothing due to connection, fallback to SQLite demo
    print("Falling back to in-memory SQLite demo (for testing).")
    return fetch_user_sqlite_demo(username)

if __name__ == "__main__":
    uname = input("Enter username: ").strip()
    try:
        user = fetch_user(uname)
    except ValueError as e:
        print("Error:", e)
        sys.exit(1)

    if user:
        print("User found:")
        print(user)
    else:
        print("No user with that username or connection failed.")


Enter username: rakshith
Error connecting to MySQL: 2003: Can't connect to MySQL server on 'localhost:3306' (Errno 111: Connection refused)
Please ensure a MySQL server is running and accessible with the provided credentials.
Falling back to in-memory SQLite demo (for testing).
No user with that username or connection failed.


TEXT-3

In [12]:
from IPython.display import HTML

html_content = """
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>Feedback Form (Demo)</title>
  <style>
    body { font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial; max-width: 680px; margin: 2rem auto; padding: 0 1rem; }
    form { display: grid; gap: .6rem; }
    label { font-weight: 600; }
    input, textarea { padding: .5rem; font-size: 1rem; width: 100%; box-sizing: border-box; }
    button { padding: .6rem 1rem; font-size: 1rem; border-radius: 6px; cursor: pointer; }
    .result { margin-top: 1rem; padding: .8rem; border: 1px solid #ddd; border-radius: 6px; background: #fafafa; }
    .muted { color: #666; font-size: .9rem; }
  </style>
</head>
<body>
  <h1>Feedback</h1>

  <form id="feedbackForm" novalidate>
    <div>
      <label for="name">Name</label><br />
      <input id="name" name="name" type="text" required minlength="2" maxlength="60" />
    </div>

    <div>
      <label for="email">Email</label><br />
      <input id="email" name="email" type="email" required />
    </div>

    <div>
      <label for="message">Message</label><br />
      <textarea id="message" name="message" rows="5" required minlength="5" maxlength="1000"></textarea>
    </div>

    <div style="display:flex; gap: .5rem; align-items:center;">
      <button type="submit">Send Feedback</button>
      <span class="muted">Your input will be displayed below (safe text rendering).</span>
    </div>
  </form>

  <section id="outputArea" aria-live="polite" class="result" hidden>
    <strong>Latest submission:</strong>
    <div id="outputContent" style="margin-top:.6rem;"></div>
  </section>

  <script>
    // Lightweight email check (HTML will also validate type="email")
    const isValidEmail = email => /^[^\n]+@[^\n]+\\.[^\n]+$/.test(email);

    const form = document.getElementById('feedbackForm');
    const outputArea = document.getElementById('outputArea');
    const outputContent = document.getElementById('outputContent');

    form.addEventListener('submit', (ev) => {
      ev.preventDefault();

      const name = form.name.value.trim();
      const email = form.email.value.trim();
      const message = form.message.value.trim();

      // Basic client-side validation (defense-in-depth)
      if (!name || name.length < 2) {
        alert('Please enter a valid name (at least 2 characters).');
        return;
      }
      if (!isValidEmail(email)) {
        alert('Please enter a valid email address.');
        return;
      }
      if (!message || message.length < 5) {
        alert('Please enter a message (at least 5 characters).');
        return;
      }

      // Use DOM methods and textContent to render user input safely (no HTML parsing)
      outputContent.innerHTML = ''; // clear previous content (we'll append nodes safely)

      const meta = document.createElement('div');
      meta.textContent = `Submitted on: ${new Date().toLocaleString()}`;

      const nameEl = document.createElement('div');
      nameEl.textContent = `Name: ${name}`;

      const emailEl = document.createElement('div');
      emailEl.textContent = `Email: ${email}`;

      const messageEl = document.createElement('div');
      // Keep message line breaks readable
      const lines = message.split(/\r?\n/);
      messageEl.appendChild(document.createTextNode('Message:'));
      const br = document.createElement('div');
      br.style.whiteSpace = 'pre-wrap';
      br.textContent = '\n' + lines.join('\n');
      // append meta + fields
      outputContent.appendChild(meta);
      outputContent.appendChild(nameEl);
      outputContent.appendChild(emailEl);
      outputContent.appendChild(messageEl);
      outputContent.appendChild(br);

      outputArea.hidden = false;

      // Optionally clear form after submit:
      // form.reset();
    });
  </script>
</body>
</html>
"""

display(HTML(html_content))

TEXT-4

In [16]:
# ✅ Secure Flask file upload demo for Google Colab (no server errors)
import os, base64, tempfile
from io import BytesIO
from uuid import uuid4
import imghdr
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
MAX_CONTENT_LENGTH = 4 * 1024 * 1024  # 4 MB
UPLOAD_DIR = os.path.join(tempfile.gettempdir(), "secure_uploads_demo")
os.makedirs(UPLOAD_DIR, mode=0o700, exist_ok=True)

app = Flask(__name__)
app.config['MAX_CONTENT_LENGTH'] = MAX_CONTENT_LENGTH
app.config['UPLOAD_DIR'] = UPLOAD_DIR

def allowed_ext(fn): return '.' in fn and fn.rsplit('.',1)[1].lower() in ALLOWED_EXTENSIONS
def detect_img(data): return imghdr.what(None, data)

@app.route("/upload", methods=["POST"])
def upload():
    if 'file' not in request.files: return jsonify({"error":"no file"}),400
    f = request.files['file']
    if f.filename == '': return jsonify({"error":"empty name"}),400

    fname = secure_filename(f.filename)
    if not allowed_ext(fname): return jsonify({"error":"bad extension"}),400
    data = f.read()
    if not detect_img(data): return jsonify({"error":"not an image"}),400

    newname = f"{uuid4().hex}.{fname.rsplit('.',1)[1].lower()}"
    path = os.path.join(app.config['UPLOAD_DIR'], newname)
    with open(path,"wb") as out: out.write(data)
    os.chmod(path,0o600)
    return jsonify({"message":"ok","stored":newname}),201

# --- Simulate uploads (no network sockets) ---
client = app.test_client()

# 1×1 transparent PNG (base64)
png_bytes = base64.b64decode(
    b"iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR4nGNgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII="
)

print("1️⃣  Upload valid PNG:")
r1 = client.post("/upload", data={'file':(BytesIO(png_bytes),'ok.png')},
                 content_type='multipart/form-data')
print(r1.status_code, r1.json)

print("\n2️⃣  Upload with malicious filename:")
r2 = client.post("/upload", data={'file':(BytesIO(png_bytes),'../../hack.sh')},
                 content_type='multipart/form-data')
print(r2.status_code, r2.json)

print("\n3️⃣  Upload non-image disguised as PNG:")
r3 = client.post("/upload", data={'file':(BytesIO(b'echo hacked'),'bad.png')},
                 content_type='multipart/form-data')
print(r3.status_code, r3.json)

print("\n✅ Files in upload dir:", os.listdir(UPLOAD_DIR))


1️⃣  Upload valid PNG:
201 {'message': 'ok', 'stored': 'd8e5cebe2e604f4a93326a8f55dfaf10.png'}

2️⃣  Upload with malicious filename:
400 {'error': 'bad extension'}

3️⃣  Upload non-image disguised as PNG:
400 {'error': 'not an image'}

✅ Files in upload dir: ['b19d5a3b7f304fbcbd8c827c80226e5c.png', 'd8e5cebe2e604f4a93326a8f55dfaf10.png']
